In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

R = 0.0083145  # kJ/(mol*K)
T = 298  # K

solvent_dict = {
    'Acetat': ('CC(=O)[O-]', 'Acetate'),
    'Acetic-acid': ('CC(=O)O', 'Acetic acid', 6.15, 'small acid'),
    'Aceton': ('CC(=O)C', 'Acetone', 20.7, 'polar aprotic'),
    'Acetonitrile': ('CC#N', 'Acetonitrile', 37.5, 'polar aprotic'),
    'Ammonia': ('N', 'Ammonia', 16.61, 'ammonia', 'polar aprotic'),
    'Ammonium': ('[NH4+]', 'Ammonium', 7.1), #ammonium chloride - solid phase. Will adjust with solution conc. 
    'Benzene': ('c1ccccc1', 'Benzene', 2.28, 'nonpolar'),
    'Benzoat': ('[O-]C(=O)c1ccccc1', 'Benzoate'),
    'Benzylacetat': ('CC(=O)OCc1ccccc1', 'Benzyl acetate', 5.34, 'polar aprotic'),
    'Butanon': ('CCC(=O)C', 'Butanone', 18.85, 'polar aprotic'),
    'Chloride': ('[Cl-]', 'Chloride', 7.1), #ammonium chloride - solid phase. Will adjust with solution conc.
    'Chloroform': ('C(Cl)(Cl)Cl', 'Chloroform', 4.8, 'nonpolar'),
    'Cyclohexan': ('C1CCCCC1', 'Cyclohexane', 2.02, 'nonpolar'),
    'Di-2-butylamin': ('CC[C@H](C)N[C@H](C)CC', 'Di-2-butylamine', 4.71, 'nonpolar'),
    'Dichlormethan': ('C(Cl)Cl', 'Dichloromethane', 8.93, 'polar aprotic'),
    'Diethanolamin': ('OCCNCCO', 'Diethanolamine', 25.75, 'polar protic'),
    'Diethanolammonium': ('OCC[NH2+]CCO', 'Diethanolammonium'),
    'Diethylenamin': ('CCNCC', 'Diethylamine', 3.8, 'nonpolar'), #Average of three sources - 3.6, 3.8, 3.92 all reported.
    'Diethylenammonium': ('CC[NH2+]CC', 'Diethylenammonium'),
    'Diethylether': ('CCOCC', 'Diethylether',4.33, 'nonpolar'),
    'Dioctylether': ('CCCCCCCCOCCCCCCC', 'Dioctylether', 2, 'nonpolar'), #Estimate
    'DMA': ('CC(=O)N(C)C', 'DMA', 37.8, 'polar aprotic'),
    'DMF': ('CN(C)C=O', 'DMF', 36.7, 'polar aprotic'),
    'DMSO': ('CS(=O)C', 'DMSO', 46.7, 'polar aprotic'),
    'EC': ('C1COC(=O)O1', 'EC', 89.78, 'polar aprotic'),
    'EMC': ('CCOC(=O)OC', 'EMC', 2.99, 'nonpolar'),
    'Ethanol': ('CCO', 'Ethanol', 24.55, 'polar protic'),
    'Ethylacetat': ('CCOC(=O)C', 'Ethyl acetate', 6.02, 'polar aprotic'),
    'Ethylenamin': ('CCN', 'Ethylenamine', 6.94),
    'Ethylenammonium': ('CC[NH3+]', 'Ethylenammonium'),
    'Ethylenglykol': ('OCCO', 'Ethylene glycol', 37.0, 'polar protic'),
    'Formiat': ('C(=O)[O-]', 'Formate'),
    'Formic-acid': ('O=CO', 'Formic acid', 51.1, 'small acid'),
    'g-Butyrolacton': ('O=C1CCCO1', 'γ-Butyrolactone', 41.68, 'polar aprotic'),
    'Glycerin': ('OCC(O)CO', 'Glycerol', 46.5, 'polar protic'),
    'H2O': ('O', 'Water', 80.1, 'water'),
    'H2SO4': ('O=S(=O)(O)O', 'Sulfuric acid', 100), #Lots of varying reports 21.9, 100, 84, 80, 106. Chosen 100 for now.
    'Hexafluorbenzol': ('Fc1c(F)c(F)c(F)c(F)c1F', 'Hexafluorobenzene',  2.05, 'nonpolar'),
    'Isooctane': ('CC(C)CC(C)(C)C', 'Isooctane', 1.94, 'nonpolar'),
    'Isopropanol': ('CC(O)C', 'Isopropanol', 17.9, 'polar protic'),
    'Methanolat': ('C[O-]', 'Methanolate'),
    'n-Hexan': ('CCCCCC', 'n-Hexane', 1.88, 'nonpolar'),
    'Nonandecanol': ('CCCCCCCCCCCCCCCCCCCO', 'Nonandecanol', 3.82, 'fatty alcohol'),
    'Octanol': ('CCCCCCCCO', 'Octanol', 10.3, 'fatty alcohol'),
    'o-Dichlorbenzol': ('Clc1ccccc1Cl', 'o-Dichlorobenzene', 9.93, 'nonpolar'),
    'Oleic-methyl-ester': ('CCCCCCCCC=CCCCCCCCC(=O)OC', 'Oleic methyl ester', 3.21, 'nonpolar'),
    'Perfluoro-hexan': ('C(C(C(C(F)(F)F)(F)F)(F)F)(C(C(F)(F)F)(F)F)(F)F', 'Perfluorohexane', 1.57, 'nonpolar'),
    'Propylenglykol': ('C[C@@H](O)CO', 'Propylene glycol', 32.0, 'polar protic'),
    'Tetraethylenammonium': ('CC[N+](CC)(CC)CC', 'Tetraethylenammonium'),
    'THF': ('O1CCCC1', 'THF', 7.58, 'polar aprotic'),
    'Toluol': ('Cc1ccccc1', 'Toluene', 2.38, 'nonpolar'),
    'Tributylphosphat': ('O=P(OCCCC)(OCCCC)OCCCC', 'Tributyl phosphate', 8.29, 'polar aprotic'),
    'Triethanolamin': ('OCCN(CCO)CCO', 'Triethanolamine', 28.11, 'polar protic'),
    'Triethanolammonium': ('OCC[NH+](CCO)CCO', 'Triethanolammonium'),
    'Triethylenamin': ('CCN(CC)CC', 'Triethylamine', 2.42, 'nonpolar'),
    'Triethylenammonium': ('CC[NH+](CC)CC', 'Triethylammonium'),
    'Triglyme': ('COCCOCCOCCOC', 'Triglyme', 7.50, 'polar aprotic'),
    'Urea': ('NC(N)=O', 'Urea', 2.9, 'urea'),
}

# https://www.engineeringtoolbox.com/liquid-dielectric-constants-d_1263.html
# https://people.chem.umass.edu/xray/solvent.html
# https://depts.washington.edu/eooptic/linkfiles/dielectric_chart%5B1%5D.pdf

solvents = list(zip(*solvent_dict.values()))[0]
solvent_names = list(zip(*solvent_dict.values()))[1]
# dielctric_constants, solvent_classes = 
# smiles_to_names_dict = {k:v for k,v in zip(solvents, solvent_names)}
# solvent_class_dict = {v[1]:v[3] for k,v in solvent_dict.items()}
# solvent_class_order = ['nonpolar', 'fatty alcohol', 'polar aprotic', 'polar protic', 'small acid', 'urea', 'water']

In [2]:
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns


def lower_triangular_heatmap(data, col_labels, row_labels, hide_diags=False, annot_fmt=".0f",
                             cbar_title="% Agreement"):
    
#     if hide_diags:
#         col_labels = col_labels[:-1]
#         row_labels = row_labels[1:]

    mask = np.zeros_like(data, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Want diagonal elements as well
    mask[np.diag_indices_from(mask)] = hide_diags

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(24, 24))

    # Generate a custom diverging colormap
#     cmap = sns.diverging_palette(220, 10, as_cmap=True)
    cmap = sns.diverging_palette(10, 220, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns_plot = sns.heatmap(data, mask=mask, cmap=cmap, annot=True,  fmt=annot_fmt,
                           yticklabels=row_labels, xticklabels=col_labels, annot_kws=dict(size=12),
                           square=True, linewidths=.5, cbar_kws=dict(shrink=.6, pad=-0.1))

    # rotate bottom tick labels
    ax.set_xticklabels(ax.get_xticklabels(),rotation=-40,ha="left",rotation_mode='anchor')
    
    # colorbar font/tick sizes and label
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=14)
    ax.figure.axes[-1].yaxis.label.set_size(12)
    cbar.ax.set_title(cbar_title, fontsize=16, pad=30)
    
    # tick sizes
    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=12)
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=12)

In [3]:
coords_df = pd.read_pickle("../data/debug/dft_coords.pkl.gz")

In [8]:
coords_df

,conf_id,mol_id,mol
0,conf00001,molecule_16112,"(Atom('F', [-2.0249201, -0.5828004, 1.7250746]..."
1,conf00002,molecule_16112,"(Atom('F', [-2.338532, -1.0670928, 1.169808], ..."
2,conf00003,molecule_16112,"(Atom('F', [-2.0065991, -0.6376301, 1.7046043]..."
3,conf00004,molecule_16112,"(Atom('F', [-1.4754949, -0.057802, 2.1413368],..."
4,conf00005,molecule_16112,"(Atom('F', [-2.075072, -0.7039253, 1.6671738],..."
...,...,...,...
22,conf00023,molecule_112040377,"(Atom('N', [-3.5346181, 3.4568772, 1.7964195],..."
23,conf00024,molecule_112040377,"(Atom('N', [-3.4805912, -1.410301, -4.6962123]..."
24,conf00025,molecule_112040377,"(Atom('N', [-3.5640049, -1.4024268, -4.6163953..."
25,conf00026,molecule_112040377,"(Atom('N', [-0.5572661, -0.87957, -5.2247276],..."


In [6]:
data_df = pd.read_pickle("../data/debug/free_energy.pkl.gz")
data_df['G(gas+solvation)'] = data_df['E(gas)'] + data_df['G(solvation)']
data_df['G(gas+RRHO)'] = data_df['E(gas)'] + data_df['G(RRHO)']
data_df

KeyError: 'E(gas)'

In [7]:
data_df

,conf_id,dE,dG,E,G,mol_id,solvent
148716,conf00002,0.000000,0.000000,NaN,NaN,11168,Acetic-acid
148717,conf00001,0.033269,0.023137,NaN,NaN,11168,Acetic-acid
148718,conf00002,0.000000,0.000000,NaN,NaN,11168,Ammonia
148719,conf00001,0.033269,0.035010,NaN,NaN,11168,Ammonia
148720,conf00002,0.000000,0.000000,NaN,NaN,11168,Benzene
...,...,...,...,...,...,...,...
175432100,conf00080,33.345186,31.259620,NaN,NaN,116792918,Urea
175432101,conf00083,33.883571,31.774157,NaN,NaN,116792918,Urea
175432102,conf00092,38.819036,37.655454,NaN,NaN,116792918,Urea
175432103,conf00113,49.667794,39.180159,NaN,NaN,116792918,Urea


In [ ]:
avail_mol_ids = data_df.mol_id.unique()

In [ ]:
pd.set_option('display.max_rows', None)
id_to_smi = pd.read_csv("../data/full/id_smiles_all.smi").set_index("ID")
id_to_smi[id_to_smi.index.isin(data_df.mol_id.unique())]

In [ ]:
mol_id = avail_mol_ids[12]
print(mol_id)
id_to_smi.loc[mol_id].SMILES

In [ ]:
for mol_id in avail_mol_ids:
    smi = id_to_smi.loc[mol_id].values[0]
    print(mol_id)
    print(smi)
    display(Chem.MolFromSmiles(smi))

### do lowest energy conformers change between solvents?

In [ ]:
solvents = data_df.solvent.unique()
n_solvents = len(solvents)
agreement, agreement_counter = np.zeros([len(solvents), len(solvents)]), 0
gas_agreement = []

dG_solution_dG_solution_pr = np.zeros([len(solvents), len(solvents)])
dE_gas_dG_solution_pr = np.zeros([len(solvents), len(solvents)])
dE_gas_dG_solv_pr = np.zeros([len(solvents), len(solvents)])
dE_gas_dG_rrho_pr = np.zeros([len(solvents), len(solvents)])
spearman_r_counter = 0
threshold = 20

n_confs_min_for_q = []
n_confs_total = []
partition_factor = 0.95

# add column for gas phase
for mol_id, df in tqdm(data_df.groupby("mol_id")):
    
    # skip if less than two conformers
    if len(df) <= (2*len(solvents)):
        continue
    
    # lowest energy conformer
    min_conf_soln = df.set_index('conf_id').groupby("solvent")['dG(solution)'].idxmin()
    agree = np.array(min_conf_soln.values == min_conf_soln.values[:, None], dtype=int)
    agreement += agree
    agreement_counter += 1
    
    # does conformer order change after g_solvation and rrho?
    min_conf_gas = df.set_index('conf_id').groupby("solvent")['dE(gas)'].idxmin()
    gas_agree = np.array(min_conf_soln.values == min_conf_gas.values, dtype=int)
    gas_agreement.append(gas_agree)
    
    # pearson correlation
    low_energy_confs = df[df['dG(solution)']<threshold].conf_id.unique()
    df_low_energy_confs = df[df['conf_id'].isin(low_energy_confs)]
    
    # skip if less than two conformers
    # mutiple columns for "values" will return cross correlations
    if len(df_low_energy_confs) > (2*len(solvents)):
        pr = df_low_energy_confs.pivot(index='conf_id', columns='solvent', \
                                       values=['dE(gas)', 'dG(solution)', 'G(gas+solvation)', 'G(gas+RRHO)']).corr(method='spearman').values
        dG_solution_dG_solution_pr += pr[1*n_solvents:(1*n_solvents+n_solvents), 1*n_solvents:(1*n_solvents+n_solvents)]
        dE_gas_dG_solution_pr += pr[0*n_solvents:(0*n_solvents+n_solvents), 1*n_solvents:(1*n_solvents+n_solvents)]
        dE_gas_dG_solv_pr += pr[0*n_solvents:(0*n_solvents+n_solvents), 2*n_solvents:(2*n_solvents+n_solvents)]
        dE_gas_dG_rrho_pr += pr[0*n_solvents:(0*n_solvents+n_solvents), 3*n_solvents:(3*n_solvents+n_solvents)]
        spearman_r_counter += 1
        
        # partition functions
        q_df = df.sort_values(['solvent', 'dG(solution)'])
        q_df['boltzmann_factor'] = np.exp(-q_df['dG(solution)'] / (R*T))
        q_df['boltzmann_factor_cum'] = q_df.groupby('solvent')['boltzmann_factor'].apply(lambda x: \
                                            np.cumsum(x) / np.cumsum(x).max())
        n_confs_min = q_df.groupby('solvent')['boltzmann_factor_cum'].apply(lambda x: \
                                            (x < partition_factor).argmin()+1).values
        n_confs_total.append(q_df.groupby('solvent')['mol_id'].count().values)
        n_confs_min_for_q.append(n_confs_min)
    
agreement = agreement/agreement_counter * 100
# gas_agreement = gas_agreement/agreement_counter * 100

dG_solution_dG_solution_pr = dG_solution_dG_solution_pr/spearman_r_counter
dE_gas_dG_solution_pr = dE_gas_dG_solution_pr/spearman_r_counter
dE_gas_dG_solv_pr = dE_gas_dG_solv_pr/spearman_r_counter
dE_gas_dG_rrho_pr = dE_gas_dG_rrho_pr/spearman_r_counter

In [ ]:
n_solvents

In [ ]:
q_data_df = pd.DataFrame({
    'solvent': solvents.tolist()*len(n_confs_total),
    'n_confs': np.stack(n_confs_total).ravel(),
    'n_confs_min': np.stack(n_confs_min_for_q).ravel(),
})

q_data_df['percent_confs_needed'] = q_data_df['n_confs_min'] / q_data_df['n_confs'] * 100



fig, ax1 = plt.subplots(figsize=(16, 9))
sns.set_style("white")

sns.boxplot(
    data=q_data_df,
    x="solvent",
    y="percent_confs_needed",
    ax=ax1,
    palette="Blues",
    showfliers=True,
)

ax1.set_xlabel("Solvent", fontsize=18, labelpad=20)
ax1.set_ylabel("% conformers needed", fontsize=18, labelpad=10)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_xticklabels(labels=solvents, fontsize=16, rotation=60, rotation_mode="default", ha="right")
ax1.tick_params(axis='x', labelsize=16)
ax1.tick_params(axis='y', labelsize=16)
ax1.legend(fontsize=16)

In [ ]:
from scipy.optimize import curve_fit

fig, ax1 = plt.subplots(figsize=(16, 9))
sns.set_style("white")

def func(x, a, c, d):
    return a*np.exp(-c*x)+d

xs, ys = [], []
for s in tqdm(solvents):
    
    # log norm the x data
    x = np.log(q_data_df[q_data_df['solvent']==s]["n_confs"].values)
    
    # fit params to exponential fit
    popt, pcov = curve_fit(func, x, q_data_df[q_data_df['solvent']==s]["percent_confs_needed"].values, 
                           p0=(1, 1e-6, 1))
    
    # apply params to fit and get y
    y = func(q_data_df[q_data_df['solvent']==s]["n_confs"].values, *popt)
    
    # plot unnormalized version on log scale
    sns.lineplot(
        x=np.exp(x),
        y=y,
        label=s,
        ax=ax1,
    )
    
ax1.set_xlabel("N conformers", fontsize=18, labelpad=20)
ax1.set_ylabel("% conformers needed", fontsize=18, labelpad=10)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.tick_params(axis='x', labelsize=16)
ax1.tick_params(axis='y', labelsize=16)
ax1.legend(fontsize=10)
    
plt.xscale('log')
plt.show()

In [ ]:
lower_triangular_heatmap(agreement, solvent_names, solvents)

In [ ]:
len(solvent_names)

In [ ]:
gas_agreement_df = pd.DataFrame({
    'solvent': solvents.tolist()*len(gas_agreement),
    'gas_agreement': np.stack(gas_agreement).ravel(),
})


fig, ax1 = plt.subplots(figsize=(16, 9))
sns.set_style("white")

sns.barplot(
    data=gas_agreement_df,
    x='solvent',
    y='gas_agreement',
    ax=ax1,
    palette="Blues",
)

ax1.set_xlabel("Solvent", fontsize=18, labelpad=20)
ax1.set_ylabel("% agreement of lowest energy conformer in gas phase", fontsize=18, labelpad=10)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_xticklabels(labels=solvents, fontsize=16, rotation=60, rotation_mode="default", ha="right")
ax1.tick_params(axis='x', labelsize=16)
ax1.tick_params(axis='y', labelsize=16)
ax1.legend(fontsize=16)

In [ ]:
lower_triangular_heatmap(dG_solution_dG_solution_pr, solvents, solvents, annot_fmt=".2f", cbar_title="Spearman")

In [ ]:
dE_gas_dG_solution_pr[0]

In [ ]:
dE_gas_dG_solv_pr[0]

In [ ]:
dE_gas_dG_rrho_pr[0]

In [ ]:
from rdkit import Chem
Chem.MolFromSmiles('CCOC(=O)OC')